In [830]:
from pycobyla.cobyla import Cobyla


class GCobyla(Cobyla):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.track = np.array((self.x,))
        
    def _add_track(self):
        if (self.optimal_vertex != self.track[-1]).all():
            self.track = np.vstack((self.track, self.optimal_vertex))
        
    def run(self):
        self.set_initial_simplex()
        self._add_track()
        
        self.ibrnch = True

        while True:
            yield 'step 1'
            self.L140_review_current_simplex()
            self._add_track()
            
            yield 'setp 2'
            state = self.L370_generate_x_start()
            self._add_track()
            if state == self.FINISH:
                break

In [831]:
import numpy as np

def gaussian(x, mu=None, sig=None, A=1):
    n = len(x)
    mu =  np.zeros(n) if mu is None else mu
    sig = np.ones(n) if sig is None else sig

    zz = ((((x - mu) / sig) ** 2) / 2).sum()
    return A * np.exp(-zz)


def neg_gaussian(x, mu=None, sig=None, A=1):
    return -gaussian(x, mu=mu, sig=sig, A=A)


def gaussian_optimizer():
    F = neg_gaussian
    c1 = lambda x: 1 - x[0]
    c2 = lambda x: 1 + x[0]
    c3 = lambda x: 1 - x[1]
    c4 = lambda x: 1 + x[1]

    C = (c1, c2, c3, c4)
    x = np.ones(2)    
    mu = np.array((0, 0))
    
    opt = GCobyla(x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt


In [832]:
%matplotlib widget
#%matplotlib notebook
import itertools

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np


def simplex_3d_plot(opt):
    plt.close('all')
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    
    points = np.array((opt.x, *(opt.sim + opt.x), opt.x))
    points = np.c_[points, np.zeros(points.shape[0])]
    ax.plot(points[0,0], points[0,1], points[0, 2], 'go')
    ax.plot(points[1:-1,0], points[1:-1,1], points[1:-1, 2], 'bo')
    ax.plot(points[...,0], points[...,1], points[..., 2], 'r-', lw=2)
    
    # Make data.
    max_p = points.max(axis=0)[:-1].max()
    min_p = points.min(axis=0)[:-1].min()

    X = np.linspace(min_p, max_p, 50)
    xv, yv = np.meshgrid(X, X)
    zv = np.array(tuple(opt.F(xy) for xy in itertools.product(X, X)))
    zv = zv.reshape(xv.shape)

    # Plot the surface.
    surf = ax.plot_surface(xv, yv, zv, cmap=cm.plasma, linewidth=0, antialiased=False)

    # Customize the z axis.
    ax.set_zlim(-1., 0.1)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    plt.show()

In [833]:
opt = gaussian_optimizer()
steps_it = opt.run()

simplex_3d_plot(opt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [868]:
next(steps_it)
simplex_3d_plot(opt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [869]:
def simplex_2d_plot(opt, target=None):
    plt.close('all')
    fig, ax = plt.subplots()
    
    points = np.array((opt.x, *(opt.sim + opt.x), opt.x))
    ax.plot(points[0,0], points[0,1], 'go')
    ax.plot(points[1:-1,0], points[1:-1,1], 'bo')
    if target:
        ax.plot(target[0], target[1], 'r*')
    ax.plot(points[...,0], points[...,1], 'r-', lw=2)
    
    # Make data
    max_p = points.max(axis=0).max()
    min_p = points.min(axis=0).min()

    X = np.linspace(max_p + (0.1 * abs(max_p)), min_p - (0.1 * abs(min_p)), 50)
    xv, yv = np.meshgrid(X, X)
    zv = np.array(tuple(opt.F(xy) for xy in itertools.product(X, X)))
    zv = zv.reshape(xv.shape)
    
    cs = ax.contourf(xv, yv, zv, cmap=cm.PuBu_r)
    cbar = fig.colorbar(cs)
    
    plt.show()


In [880]:
opt = gaussian_optimizer()
steps_it = opt.run()

simplex_2d_plot(opt, target=(0,0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [946]:
next(steps_it)
simplex_2d_plot(opt, target=(0,0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [947]:
%matplotlib widget

import matplotlib.pyplot as plt

opt = gaussian_optimizer()
steps_it = opt.run()

In [1125]:
n_points = 5
target = np.zeros(2)

next(steps_it)
plt.close('all')
fig, ax = plt.subplots(figsize=(6, 6))
track = opt.track[-n_points:]
ax.plot(track[:, 0], track[:, 1], 'ro:')

link_best = np.array((track[-1], target))
ax.plot(link_best[:, 0], link_best[:, 1], 'r-')
ax.plot(target[0], target[1], 'g*')

plt.show()

StopIteration: 

In [1197]:
n = 0
plt.close('all')
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(opt.track[n:, 0], opt.track[n:, 1], 'ro:')

link_best = np.array((opt.track[-1], target))
ax.plot(link_best[:, 0], link_best[:, 1], 'r-')
ax.plot(target[0], target[1], 'g*')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …